In [1]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex

import import_ipynb
import data_utils as data
tt = gp.Model('IIITB Exams Timetable')

Restricted license - for non-production use only - expires 2026-11-23


## Make one binary variable for each room-course-timeslot-day combination
$$
E_{crtd} =
\begin{cases} 
1 & \text{if a course $c$ exam is scheduled in room $r$ at time $t$ on day $d$,} \\
0 & \text{Otherwise.}
\end{cases}
$$

In [2]:
E = tt.addVars(
    [
        (c,r,t,d)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
        for t in range(data.n_times)
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)

## Hard Constraints

In [3]:
# Exactly one slot for each course
# sum of enrolments in courses assigned to a room <= room capacity
# no clash for any student
# all slots for a course on any day are contiguous
# exam lengths for each course


# ---Modified Constraints--- (Sarvesh)
# Exactly one continuous slot for each course satisying the length of the course
# Exactly one room for each course
# Exactly one day for each course

## Hard Constraint 1: Exactly one continuous slot for each course satisying the length of the course (Sarvesh)

$$
\sum_{r=1}^{\text{data.n\_rooms}}\sum_{d=1}^{\text{data.n\_days}} \sum_{t=1}^{\text{data.n\_times} - \text{exam\_slots}(c) + 1} \sum_{t' = t}^{t + \text{exam\_slots}(c) - 1} 
\text{feasibility\_exam\_slot}(\text{times}[t], \text{times}[t + \text{exam\_slots}(c) - 1], c) \cdot E_{crt'd} = \text{exam\_slots}(c), 
$$


$$
\sum_{r=1}^{\text{data.n\_rooms}}\sum_{d=1}^{\text{data.n\_days}} \sum_{t=1}^{\text{data.n\_times}} E_{crtd} = \text{exam\_slots}(c), 
$$

$$
\forall c \in \{1, \dots, \text{data.n\_courses}\}.
$$




In [4]:
# Hard Constraint 1: Exactly one continuous slot for each course satisying the length of the course (Sarvesh)


# ------------------------ Exam duration of a course doesn't exceed time slot allocated for the course------------------------
def exam_length(course:int):
    return data.get_exam_duration(course)
    #returns the exam length of the course given the course id (in the 'course' dataframe)

def exam_slots(course:int):
    return (data.get_exam_duration(course)//15) + 1
    #returns the number of slots required for the course given the course id (in the 'course' dataframe)


from datetime import datetime
def time_to_minutes(time_str):
    """Convert time string to minutes since midnight."""
    time_obj = datetime.strptime(time_str, '%H:%M')
    return time_obj.hour * 60 + time_obj.minute

def feasibility_exam_slot(start, end, course):
    """Returns True if the start and end time is feasible for the course given the course id."""
    start_minutes = time_to_minutes(start)
    end_minutes = time_to_minutes(end)
    exam_length = exam_length(course)
    # Check if the time difference matches the exam length
    return (end_minutes - start_minutes)== exam_length

#read as for each course c, for each room r, for each day d, for each time t in the day (t to t+exam_slots(c)), the sum of E[(c,r,t,d)] is equal to exam_slots(c)
for c in range(data.n_courses):
    tt.addConstr(
        gp.quicksum(
            feasibility_exam_slot(data.times[t],data.times[t+exam_slots(c)-1],c) * E[(c, r, t_, d)]
            for r in range(data.n_rooms)
            for d in range(data.n_days)
            for t_ in range(t, t + exam_slots(c))
            for t in range(data.n_times - exam_slots(c) + 1)
        ) == exam_slots(c),
        name=f"Course {c} continuous slots"
    )

for c in range(data.n_courses):
    tt.addConstr(
        gp.quicksum(
            E[(c, r, t, d)] 
            for r in range(data.n_rooms) 
            for t in range(data.n_times) 
            for d in range(data.n_days)
        ) == exam_slots(c),
        name=f"Course {c} slots overall"
    )


## Hard Constraint 2: Sum of enrolments in courses assigned to a room <= room capacity (Sarvesh)

$$
\sum_{c=1}^{\text{data.n\_courses}} \text{enrollments\_in\_course}(c) \cdot E_{crtd} \leq \text{room\_capacity}(r)
$$

$$
\forall r \in \{1, \dots, \text{data.n\_rooms}\}, \forall d \in \{1, \dots, \text{data.n\_days}\}, \forall t \in \{1, \dots, \text{data.n\_times}\}.
$$

In [5]:
# Hard Constraint 2: Sum of enrolments in courses assigned to a room <= room capacity (Sarvesh)

def enrollments_in_course(course:int):
    return data.get_length_course_taken_by_students(course)
    #returns the number of enrolments in the course given the course id (in the 'course' dataframe)

def room_capacity(room:int):
    return data.get_room_capacity(room)
    #returns the capacity of the room given the room id (in the 'room' dataframe)

# read as for each room r, day d, time t, sum of enrolments in courses assigned to the room <= room capacity
for r in range(data.n_rooms):
    for d in range(data.n_days):
        for t in range(data.n_times):
            tt.addConstr(
                gp.quicksum(
                    enrollments_in_course(c) * E[(c, r, t, d)]
                    for c in range(data.n_courses)
                ) <= room_capacity(r),
                name=f"Room {r} day {d} time {t} capacity"
            )

## Hard Constraint 3: No Clash for any student (Sarvesh)

$$
\sum_{\substack{c=1 \\ \text{if student $s$ is taking course $c$}}}^{\text{data.n\_courses}} 
\sum_{r=1}^{\text{data.n\_rooms}} E_{crtd} \leq 1,
$$

$$
\forall s \in \{1, \dots, \text{data.n\_students}\}, \forall d \in \{1, \dots, \text{data.n\_days}\}, \forall t \in \{1, \dots, \text{data.n\_times}\}.
$$

In [6]:
# Hard Constraint 3: No clash for any student (Sarvesh)

def check_student_taking_course(student:int,course:int):
    #returns True if the student is taking the course given the student id and course id (in the 'student' and 'course' dataframe)
    return data.check_student_taking_course(student,course)

for s in range(data.n_students):
    for d in range(data.n_days):
        for t in range(data.n_times):
            tt.addConstr(
                gp.quicksum(
                    E[(c, r, t, d)]
                    for c in range(data.n_courses)
                    for r in range(data.n_rooms)
                    if check_student_taking_course(s, c)
                ) <= 1,
                name=f"Student {s} day {d} time {t} clash"
            )

## Hard Constraint 4: Exactly one room for a course (Sarvesh)

$$
\sum_{r=1}^{\text{data.n\_rooms}} 
\text{signum}\left(
\sum_{d=1}^{\text{data.n\_days}} 
\sum_{t=1}^{\text{data.n\_times}} 
E_{crtd}
\right) = 1,
$$

$$
\forall c \in \{1, \dots, \text{data.n\_courses}\}.
$$


In [7]:
# Hard Constraint 4: Exactly one room for each course (Sarvesh)

def signum(x):
    if x>0:
        return 1
    return 0

for c in range(data.n_courses):
    tt.addConstr(
        gp.quicksum(
            # E[(c, r, t, d)]
            signum(gp.quicksum(
                E[(c, r, t, d)]
                for d in range(data.n_days)
                for t in range(data.n_times)
                )
            )
            for r in range(data.n_rooms)
        ) == 1,
        name=f"Course {c} room"
    )

## Hard Constraint 5: Exactly one day for a course (Sarvesh)

$$
\sum_{r=1}^{\text{data.n\_days}} 
\text{signum}\left(
\sum_{d=1}^{\text{data.n\_rooms}} 
\sum_{t=1}^{\text{data.n\_times}} 
E_{crtd}
\right) = 1,
$$

$$
\forall c \in \{1, \dots, \text{data.n\_courses}\}.
$$


In [8]:
# Hard Constraint 5: Exactly one day for each course (Sarvesh)


def signum(x):
    if x>0:
        return 1
    return 0

for c in range(data.n_courses):
    tt.addConstr(
        gp.quicksum(
            # E[(c, r, t, d)]
            signum(gp.quicksum(
                E[(c, r, t, d)]
                for r in range(data.n_rooms) 
                for t in range(data.n_times)
                )
            )
            for d in range(data.n_days)
        ) == 1,
        name=f"Course {c} day"
    )

## Soft Constraints

In [9]:
# preferred time slots for exams

## Objective Function

In [10]:
# Maximize Gaps between exams for a student / batch